# Building the Sedimentation Tank Property Model
## Part 1: The Parameter Block

**Sedimentation tanks** utilize one of the most basic water treatment processes: settling. Turbid water (water + suspended solids) enters the unit and gravity-based separation produces an overflow stream with clear water as well as an underflow stream with settled sludge.

---
**Property models** are used to describe the components and properties of a given stream and have a 3-tiered structure:
1. **Parameter Block - "Instruction Manual"**
   - Defines the components in the stream (H2O, TSS, etc.)
   - Creates parameters (density, molecular weight, etc.)

2. **State Block Class - "Model Initializer"**
   - Defines how to prime a model so that it's readily solvable
  
3. **State Block Data - "Calculator"**
   - Creates variables
   - Defines expressions and constraints

---

In this tutorial, you will:
- Understand the basic structure of WaterTAP property models
- Build a working property model


**Goal**: Create a property model with just 2 components: Water and Suspended Solids

**Time**: 10-15 minutes


## Step 1: Import Modules

In [1]:
# Imports from Pyomo
from pyomo.environ import (
    Constraint,
    Var,
    Param,
    NonNegativeReals,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import (
    declare_process_block_class,
    PhysicalParameterBlock,
    StateBlockData,
    StateBlock,
)
from idaes.core.base.components import Component, Solvent
from idaes.core.base.phases import LiquidPhase

## Step 2: Define the Parameter Block

This is the "Instruction Manual" for our sedimentation system that defines the components and sets parameters.

<details>
  <summary>Click the arrow for a hint for TODO #1!</summary>
    
The syntax for defining the liquid phase is: `self.Name = LiquidPhase()`. Remember to replace `Name` with an abbreviation for the phase.
</details>

<details>
  <summary>Click the arrow for a hint for TODO #2!</summary>
    
The syntax for defining components is: `self.ComponentName = Solvent()` or `self.ComponentName = Component()`
</details>

<details>
  <summary>Click the arrow for a hint for TODO #3!</summary>
    
The syntax for defining density is: `self.dens_mass = Param(initialize=..., units=...)`
</details>

In [2]:
@declare_process_block_class("SedimentationParameterBlock")
class SedimentationParameterData(PhysicalParameterBlock):
    """
    Property model for sedimentation tank
    Components: Water (H2O) and Suspended Solids
    """
    
    CONFIG = PhysicalParameterBlock.CONFIG()

    def build(self):
        super(SedimentationParameterData, self).build()
        
        # This tells IDAES which StateBlock class to use
        self._state_block_class = SedimentationStateBlock
        
        # ============================================
        # TODO 1: Define the liquid phase
        # ============================================
        
        self.Liq = LiquidPhase()
        
        # ============================================
        # TODO 2: Define the components
        # ============================================

        # Define water
        self.H2O = Solvent()
        # Define TSS
        self.TSS = Component()
        
        # ============================================
        # TODO 3: Add a density parameter
        # ============================================
        # Wastewater is mostly water, so let's use 1000 kg/m³
        
        self.dens_mass = Param(
            initialize=1000, 
            units=pyunits.kg/pyunits.m**3,
            doc="Density of wastewater (approximately water density)"
        )
        
        # Set scaling factors (for the solver)
        self.set_default_scaling("temperature", 1e-2)
        self.set_default_scaling("pressure", 1e-6)
        self.set_default_scaling("dens_mass_phase", 1e-3, "Liq")
    
    @classmethod
    def define_metadata(cls, obj):
        """Tell IDAES what properties we support"""
        obj.add_properties({
            "flow_mass_phase_comp": {"method": None},
            "temperature": {"method": None},
            "pressure": {"method": None},
            "mass_frac_phase_comp": {"method": "_mass_frac_phase_comp"},
            "flow_vol_phase": {"method": "_flow_vol_phase"},
        })
        
        obj.add_default_units({
            "time": pyunits.s,
            "length": pyunits.m,
            "mass": pyunits.kg,
            "temperature": pyunits.K,
        })


## Step 3: Define the State Block Classes

This boilerplate code handles initialization and constructs constraints.

In [3]:
class _SedimentationStateBlock(StateBlock):
    """Simple initialization routine"""
    
    def initialize(self, *args, **kwargs):
        # For this simple model, we don't need complex initialization
        pass


@declare_process_block_class("SedimentationStateBlock", block_class=_SedimentationStateBlock)
class SedimentationStateBlockData(StateBlockData):
    """Creates variables and constraints"""
    
    def build(self):
        super(SedimentationStateBlockData, self).build()
        
        # ============================================
        # State Variables (the inputs to our model)
        # ============================================
        
        self.flow_mass_phase_comp = Var(
            self.params.phase_list,
            self.params.component_list,
            initialize=0.1,  # Start with a reasonable guess
            bounds=(0, 100),
            domain=NonNegativeReals,
            units=pyunits.kg / pyunits.s,
            doc="Mass flow rate of each component",
        )
        
        self.temperature = Var(
            initialize=298.15,  # Room temp in Kelvin
            bounds=(273.15, 400),
            domain=NonNegativeReals,
            units=pyunits.K,
            doc="Temperature",
        )
        
        self.pressure = Var(
            initialize=101325,  # Atmospheric pressure
            bounds=(1e5, 5e7),
            domain=NonNegativeReals,
            units=pyunits.Pa,
            doc="Pressure",
        )
    
    # ============================================
    # Calculated Properties (on-demand)
    # ============================================
    
    def _mass_frac_phase_comp(self):
        """Calculate mass fraction of each component"""
        self.mass_frac_phase_comp = Var(
            self.params.phase_list,
            self.params.component_list,
            initialize=0.5,
            bounds=(0, 1),
            units=pyunits.dimensionless,
            doc="Mass fraction",
        )
        
        def rule_mass_frac(b, j):
            return b.mass_frac_phase_comp["Liq", j] == b.flow_mass_phase_comp["Liq", j] / sum(
                b.flow_mass_phase_comp["Liq", jj] for jj in b.params.component_list
            )
        
        self.eq_mass_frac = Constraint(
            self.params.component_list, rule=rule_mass_frac
        )
    
    def _flow_vol_phase(self):
        """Calculate volumetric flow rate"""
        self.flow_vol_phase = Var(
            self.params.phase_list,
            initialize=1e-3,
            bounds=(0, None),
            units=pyunits.m**3 / pyunits.s,
            doc="Volumetric flow rate",
        )
        
        def rule_flow_vol(b):
            return b.flow_vol_phase["Liq"] == sum(
                b.flow_mass_phase_comp["Liq", j] for j in b.params.component_list
            ) / b.params.dens_mass
        
        self.eq_flow_vol = Constraint(rule=rule_flow_vol)
    
    def define_state_vars(self):
        """Tell IDAES what the state variables are"""
        return {
            "flow_mass_phase_comp": self.flow_mass_phase_comp,
            "temperature": self.temperature,
            "pressure": self.pressure,
        }


## Step 4: Test the Model

Let's create a turbid wastewater stream and inspect our model.

In [4]:
from pyomo.environ import ConcreteModel
from idaes.core import FlowsheetBlock

# Create a model
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Add our sedimentation property package
m.fs.properties = SedimentationParameterBlock()

# Create a state block (represents a stream of wastewater)
m.fs.stream = m.fs.properties.build_state_block([0])

# Set some values (typical primary influent)
m.fs.stream[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.99)  # 0.99 kg/s water (~1000 L/day)
m.fs.stream[0].flow_mass_phase_comp["Liq", "TSS"].fix(0.01)  # 0.01 kg/s TSS (~250 mg/L)
m.fs.stream[0].temperature.fix(273.15 + 20)  # 20°C
m.fs.stream[0].pressure.fix(101325)  # Atmospheric

# Calculate mass fractions
m.fs.stream[0].mass_frac_phase_comp

# Calculate volumetric flow
m.fs.stream[0].flow_vol_phase

print("\n" + "="*50)
print("SUCCESS! Your sedimentation property model works!")
print("="*50)
print(f"\n Wastewater Stream Properties:")
print(f"   Water flow:     {m.fs.stream[0].flow_mass_phase_comp['Liq', 'H2O'].value:.3f} kg/s")
print(f"   TSS flow:       {m.fs.stream[0].flow_mass_phase_comp['Liq', 'TSS'].value:.3f} kg/s")
print(f"   Temperature:    {m.fs.stream[0].temperature.value - 273.15:.1f}°C")
print(f"   Pressure:       {m.fs.stream[0].pressure.value:.0f} Pa")
print(f"\n Calculated Properties:")
print(f"   Water fraction: {m.fs.stream[0].mass_frac_phase_comp['Liq', 'H2O'].value:.1%}")
print(f"   TSS fraction:   {m.fs.stream[0].mass_frac_phase_comp['Liq', 'TSS'].value:.1%}")
print(f"   Flow rate:      {m.fs.stream[0].flow_vol_phase['Liq'].value * 86400:.1f} m³/day")


SUCCESS! Your sedimentation property model works!

 Wastewater Stream Properties:
   Water flow:     0.990 kg/s
   TSS flow:       0.010 kg/s
   Temperature:    20.0°C
   Pressure:       101325 Pa

 Calculated Properties:
   Water fraction: 50.0%
   TSS fraction:   50.0%
   Flow rate:      86.4 m³/day


### Summary
In this tutorial, we learned how to create a working property model by:
 
- Defining components (Water, TSS)  
- Setting parameters (density)
- Calculating properties with constraints

**This structure is IDENTICAL to the Week 4 Filtration property model, just simplified!**

The only differences:
- Week 4: H2O, NaCl, TSS → Sedimentation: H2O, TSS
- Week 4: More properties → Sedimentation: Fewer properties
- Week 4: Complex density equation → Sedimentation: Simple parameter

Consider visiting part 2 of this tutorial: Building the Sedimentation Tank Unit Model